In [1]:
import warnings

import pandas as pd
import numpy as np

warnings.filterwarnings(action='ignore')

In [2]:
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')
pitcher

,ID,선수명,데뷔년도,생년월일,팀명,연도,평균자책점,경기,승리,패배,...,땅볼/뜬공,인플레이타구타율,투구수/경기,투구수/이닝,9이닝당 삼진,9이닝당 볼넷,삼진/볼넷,피출루율,피장타율,피OPS
0,90312,가내영,NaN,1971-03-29,KIA,2002,2.52,13,1,0,...,1.25,0.357,31.4,16.3,6.48,2.88,2.25,0.368,0.423,0.791
1,72627,강철민,1998.0,1979-07-22,KIA,2002,5.68,26,5,6,...,1.05,0.346,47.5,15.9,5.33,2.55,2.09,0.363,0.478,0.841
2,92404,곽채진,NaN,1973-03-01,KIA,2002,4.3,10,0,0,...,0.42,0.190,21.3,14.5,4.91,1.23,4.00,0.276,0.444,0.720
3,98617,곽현희,NaN,1973-08-06,KIA,2002,4.13,25,2,0,...,1.56,0.281,22.0,16.9,7.44,3.86,1.93,0.345,0.45,0.795
4,71661,김주철,NaN,1982-05-05,KIA,2002,4.42,12,0,0,...,0.89,0.308,24.7,16.1,7.85,2.45,3.20,0.307,0.324,0.631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552,77318,장시환,2007.0,1987-11-01,한화,2020,5.02,26,4,14,...,1.25,0.319,93.6,18.3,7.8,5.02,1.55,0.368,0.415,0.783
3553,50706,장웅정,2020.0,1997-04-11,한화,2020,3.52,2,0,1,...,0.89,0.292,78.0,20.3,5.87,7.04,0.83,0.371,0.276,0.647
3554,74857,정우람,2004.0,1985-06-01,한화,2020,4.8,50,3,5,...,0.51,0.355,18.1,16.7,8.78,1.99,4.42,0.332,0.405,0.737
3555,50726,한승주,2020.0,2001-03-17,한화,2020,16.2,1,0,0,...,0.5,0.400,36.0,21.6,10.8,10.8,1.00,0.500,0.857,1.357


# 1. 주루데이터 붙이기
 - 주루데이터에 간혹 투수가 존재하는 경우가 있다. 이 경우를 필터링하기 위해 left_join
 - 주루는 타자에만 해당되는 지표라고 판단하여, 타자에게만 merge한다.

In [3]:
# 전체 주루데이터 불러오기
for year in range(2020, 2001, -1):

    # 데이터 불러오기
    run = pd.read_csv(f'선수데이터/주루/주루정보_팀전체({year}).csv')

    # 간단한 전처리 (연도변수추가 및 변수명 변경)
    run['연도'] = year
    run.columns = ['선수명', 'ID', '팀명', '경기(주루)', '도루시도(주루)', '도루허용(주루)', '도루실패(주루)', '도루성공률(주루)', '주루사(주루)', '견제사(주루)', '연도']
    run = run[['ID', '선수명', '팀명', '연도', '경기(주루)', '도루시도(주루)', '도루허용(주루)', '도루실패(주루)', '도루성공률(주루)', '주루사(주루)', '견제사(주루)']]
    run['도루성공률(주루)'] = run['도루성공률(주루)'].replace('-',0).astype('float')

    # 정렬
    run = run.sort_values(by=['팀명','선수명']).reset_index(drop=True)

    # 최종저장
    if year == 2020:
        run_final = run.copy()
    else:
        run_final = pd.concat([run_final, run]).reset_index(drop=True)

In [4]:
# 팀명 수정

def change_teamname2(x):  # KBO데이터
    
    if (x == '히어로즈') or (x == '넥센') or (x == '키움') or (x == '우리'):
        x = '우리/히어로즈/넥센/키움'
        
    return x

run_final['팀명'] = run_final['팀명'].apply(lambda x: change_teamname2(x))

In [5]:
# Merge
hitter[['ID', '연도']] = hitter[['ID', '연도']].astype('str')
run_final[['ID', '연도']] = run_final[['ID', '연도']].astype('str')

hitter = pd.merge(hitter, run_final, how='left', on=['ID', '선수명', '팀명', '연도'])
hitter

,ID,선수명,데뷔년도,생년월일,팀명,연도,타율,경기,타석,타수,...,순수장타율,추정득점,GPA,경기(주루),도루시도(주루),도루허용(주루),도루실패(주루),도루성공률(주루),주루사(주루),견제사(주루)
0,71610,김경언,2001.0,1982-12-07,KIA,2002,0.263,107,323,274,...,0.077,39.0,0.251,107,15,11,4,73.3,0,0
1,71609,김민철,2001.0,1982-03-14,KIA,2002,0.261,58,53,46,...,0.022,4.5,0.221,58,1,0,1,0.0,0,0
2,70612,김상훈,1996.0,1977-10-27,KIA,2002,0.221,113,391,357,...,0.092,28.6,0.198,113,4,3,1,75.0,0,0
3,90811,김인철,NaN,1971-07-14,KIA,2002,0.235,37,53,51,...,0.137,4.7,0.206,37,1,1,0,100.0,0,0
4,96616,김종국,1996.0,1973-09-14,KIA,2002,0.287,133,595,527,...,0.106,82.1,0.262,133,59,50,9,84.7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234,50707,최인호,2020.0,2000-01-30,한화,2020,0.236,47,135,127,...,0.000,7.4,0.176,47,0,0,0,0.0,0,0
4235,78288,최재훈,2008.0,1989-08-27,한화,2020,0.301,126,389,339,...,0.083,49.2,0.268,126,0,0,0,0.0,3,0
4236,74729,최진행,2004.0,1985-08-17,한화,2020,0.234,84,240,214,...,0.136,23.2,0.229,84,0,0,0,0.0,0,0
4237,62700,하주석,2012.0,1994-02-25,한화,2020,0.286,72,281,262,...,0.065,29.7,0.233,72,5,4,1,80.0,3,0


# 2. 수비데이터 붙이기
 - 수비 이닝 1 1/3과 같은 경우 변환 필요
 - (1) 포지션부터 확실히 (투수면, 타자 수비 데이터는 삭제 // 타자면 투수 수비 데이터는 삭제)
 
 - (2) 타자는 최다이닝포지션 할당, 할당하면서 각 지표의 합 또는 가중합 산출 (대표 포지션은 최다이닝포지션으로 할당 (만약 개수가 같다면 임의 할당))
 - (3) 투수지표와 타자지표를 구분 (변수구분)
 - (4) Merge가 안된 것은 조사 결과 타자에서만 존재하였고, 지명타자나 대타로만 출장하여 수비의 기록이 없는 경우였다. 이에 대해 결측치를 포지션은 수비기록없음으로, 그 외 지표들은 0을 넣었다. 

In [6]:
# 수비이닝 형변환 함수

def inning_change(value):
    
    tmp = value.split(' ')
    
    if len(tmp) == 2:
        if tmp[1] == '1/3':
            output = int(tmp[0]) + 1/3
        elif tmp[1] == '2/3':
            output = int(tmp[0]) + 2/3
            
    else: 
        if '1/3' == tmp[0]:
            output = 1/3
        elif '2/3' == tmp[0]:
            output = 2/3
        else:
            output = int(tmp[0])
        
    return output

In [7]:
for year in range(2020, 2001, -1):

    # 데이터 불러오기
    defense = pd.read_csv(f'선수데이터/수비/수비정보_팀전체({year}).csv')

    # 간단한 데이터 형변환
    defense['도루저지율'] = defense['도루저지율'].replace('-',0).astype('float')
    defense['수비율'] = defense['수비율'].replace('-',0).astype('float')
    defense['수비이닝'] = defense['수비이닝'].apply(lambda x: inning_change(x))

    # 메인포지션 할당
    for ID in defense['ID'].unique():
        tmp = defense[defense['ID'] == ID].reset_index(drop=True)
        main_position = tmp.loc[tmp['수비이닝'] == tmp['수비이닝'].max(), '포지션'].values[0]
        defense.loc[defense['ID'] == ID, '포지션'] = main_position

    # 스탯들 가중합
    
    """
    도루저지율, 수비율 제외 모두 sum
    도루저지율, 수비율은 이닝기준 가중합
    """
    
    defense2 = defense.copy()
    defense2_stat = defense2.groupby(['ID', '선수명']).sum().drop(['수비율','도루저지율'], axis=1).reset_index()
    tmp = defense2.drop_duplicates(subset=['ID','선수명'])[['ID','선수명']].reset_index(drop=True)

    for idx in range(tmp.shape[0]): ### 수비율과 도루저지율은 가중합
        ID, name = tmp.loc[idx, 'ID'], tmp.loc[idx, '선수명']
        inning = np.array(defense2.loc[(defense2['선수명'] == name) & (defense2['ID'] == ID), '수비이닝'])
        subi_rate = np.array(defense2.loc[(defense2['선수명'] == name) & (defense2['ID'] == ID), '수비율'])
        doru_rate = np.array(defense2.loc[(defense2['선수명'] == name) & (defense2['ID'] == ID), '도루저지율'])

        defense2_stat.loc[(defense2_stat['선수명'] == name) & (defense2_stat['ID'] == ID), '수비율'] = np.dot(inning, subi_rate) / np.sum(inning)
        defense2_stat.loc[(defense2_stat['선수명'] == name) & (defense2_stat['ID'] == ID), '도루저지율'] = np.dot(inning, doru_rate) / np.sum(inning)

    defense2_stat.loc[defense2_stat['수비이닝'] == 0, '수비율'] = 0
    defense2_stat.loc[defense2_stat['수비이닝'] == 0, '도루저지율'] = 0

    defense4 = pd.merge(defense2.drop_duplicates(subset=['ID'])[['ID', '선수명', '팀명', '포지션']], defense2_stat, how='left', on=['ID','선수명'])
    defense4['연도'] = year
    defense4 = defense4[['ID', '선수명', '팀명', '연도', '포지션', '경기', '선발경기', '수비이닝', '실책', '견제사', '자살', '보살', '병살', '포일', '도루허용', '도루실패', '수비율', '도루저지율']]

    # Feature 구분
    pitcher_r = defense4[defense4['포지션'] == '투수'].drop(['도루허용','도루실패','도루저지율','포일'], axis=1).reset_index(drop=True)
    hitter_r = defense4[defense4['포지션'] != '투수'].reset_index(drop=True)

    pitcher_r.columns = ['ID', '선수명', '팀명', '연도', '포지션(수비)', '경기(수비)', '선발경기(수비)', '수비이닝(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '수비율(수비)']
    hitter_r.columns = ['ID', '선수명', '팀명', '연도', '포지션(수비)', '경기(수비)', '선발경기(수비)', '수비이닝(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)', '도루실패(수비)', '수비율(수비)', '도루저지율(수비)']

    pitcher_r = pitcher_r.sort_values(by=['팀명','선수명'])
    hitter_r = hitter_r.sort_values(by=['팀명','선수명'])

    # 최종저장
    if year == 2020:
        pitcher_r_final = pitcher_r.copy()
        hitter_r_final = hitter_r.copy()

    else:
        pitcher_r_final = pd.concat([pitcher_r_final, pitcher_r]).reset_index(drop=True)
        hitter_r_final = pd.concat([hitter_r_final, hitter_r]).reset_index(drop=True)

In [8]:
# Merge
hitter[['ID', '연도']] = hitter[['ID', '연도']].astype('str')
pitcher[['ID', '연도']] = pitcher[['ID', '연도']].astype('str')
hitter_r_final[['ID', '연도']] = hitter_r_final[['ID', '연도']].astype('str')
pitcher_r_final[['ID', '연도']] = pitcher_r_final[['ID', '연도']].astype('str')

hitter_r_final['팀명'] = hitter_r_final['팀명'].apply(lambda x: change_teamname2(x))
pitcher_r_final['팀명'] = pitcher_r_final['팀명'].apply(lambda x: change_teamname2(x))

hitter = pd.merge(hitter, hitter_r_final, how='left', on=['ID', '선수명', '팀명', '연도'])
pitcher = pd.merge(pitcher, pitcher_r_final, how='left', on=['ID', '선수명', '팀명', '연도'])

In [9]:
# 타자는 결측치 109개 존재, 투수는 결측치 없음
na_cols = ['경기(수비)', '선발경기(수비)', '수비이닝(수비)', '실책(수비)',
       '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)',
       '도루실패(수비)', '수비율(수비)', '도루저지율(수비)']

hitter['포지션(수비)'] = hitter['포지션(수비)'].fillna('수비기록없음')
hitter[na_cols] = hitter[na_cols].fillna(0)

In [10]:
# 저장
hitter.to_csv('선수데이터(전처리완료)/타자.csv', index=False, encoding='utf-8-sig')
pitcher.to_csv('선수데이터(전처리완료)/투수.csv', index=False, encoding='utf-8-sig')